In [1]:
import configparser
import os
import sys
from time import sleep
from uuid import uuid4

import numpy as np

import GCode
import GRBL

#

# Code:

In [2]:
# Poplar 1x4". Cut
BlockHeight = 68.0  # [mm]
BlockLength = 143  # [mm]
BlockThickness = 19.0  # [mm]
print("{} x {} x {}".format(BlockHeight, BlockLength, BlockThickness))


68.0 x 143 x 19.0


In [3]:
config = configparser.ConfigParser()
config["GRBL"] = dict()
config["GRBL"]["port"] = "/dev/cnc_3018"
cnc = GRBL.GRBL(**config["GRBL"])

cnc.reset()

(0, ['', "Grbl 1.1f ['$' for help]", 'ok'])

In [4]:
if "Alarm" in cnc.status:
    cnc.home()

In [5]:
cnc.init()

True

# Generate Program


In [6]:
grid_spacing = 1 # [mm]

In [7]:
Xs = np.arange(grid_spacing, BlockLength, grid_spacing)
Ys = np.arange(grid_spacing, BlockHeight, grid_spacing)

In [8]:
Xs

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142])

In [9]:
Ys

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
       14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26.,
       27., 28., 29., 30., 31., 32., 33., 34., 35., 36., 37., 38., 39.,
       40., 41., 42., 43., 44., 45., 46., 47., 48., 49., 50., 51., 52.,
       53., 54., 55., 56., 57., 58., 59., 60., 61., 62., 63., 64., 65.,
       66., 67.])

In [10]:
len(Xs)

142

In [11]:
Zs=np.array([-5, 0, 5])+-14
Zs

array([-19, -14,  -9])

In [12]:
n_powers = int(len(Xs)/len(Zs))
powers = np.linspace(50, 255, n_powers, dtype=np.uint8)
powers

array([ 50,  54,  58,  63,  67,  72,  76,  81,  85,  90,  94,  99, 103,
       107, 112, 116, 121, 125, 130, 134, 139, 143, 148, 152, 156, 161,
       165, 170, 174, 179, 183, 188, 192, 197, 201, 205, 210, 214, 219,
       223, 228, 232, 237, 241, 246, 250, 255], dtype=uint8)

In [13]:
dwells = [0.01, 0.05, 0.10, 0.50, 1, 2, 2.5, 5]

In [14]:
len(dwells)

8

In [15]:
for idx3, dwell in enumerate(dwells):
    for idx1, Z in enumerate(Zs):
        for idx2, power in enumerate(powers):
            idx = (idx1*len(powers)+idx2)
            X=Xs[idx]
            Y=Ys[idx3]     

In [16]:
def init(**kwargs):
    program = GCode.GCode(**kwargs)
    program.G0(F=500)
    program.G1(F=500)
    program.G21() # Metric Units
    program.G90() # Absolute positioning.
    return program

In [17]:
def dot(X=10, Y=10, Z=0, power=1, dwell=1, **kwargs):
    program = init(**kwargs)
    
    program.G0(X=X, Y=Y, Z=Z)
    program.M3(S=power)
    program.G1(Z=Z, Y=Y, X=X)
    program.G4(P=dwell)
    program.M5()  # Laser. Off.

    return program

In [18]:
programs=list()
for idx3, dwell in enumerate(dwells):
    for idx1, Z in enumerate(Zs):
        for idx2, power in enumerate(powers):
            idx = (idx1*len(powers)+idx2)
            X=Xs[idx]
            Y=Ys[idx3]
            cfg = {
                "X": X,
                "Y": Y,
                "Z": Z,
                "power": power,
                "dwell": dwell,
                "machine": cnc
            }
            program = dot(**cfg)
            programs.append(program)

In [19]:
idx

140

In [20]:
idx3

7

In [21]:
len(Zs)

3

In [22]:
len(powers)

47

In [23]:
len(Ys)

67

In [24]:
len(Xs)

142

In [25]:
len(Xs)

142

In [26]:
powers= [100, 200, 255]

In [27]:
n_Zs = int(len(Xs)/len(powers))
Zs = np.linspace(-28.0, 0.0, n_Zs)
Zs

array([-28.        , -27.39130435, -26.7826087 , -26.17391304,
       -25.56521739, -24.95652174, -24.34782609, -23.73913043,
       -23.13043478, -22.52173913, -21.91304348, -21.30434783,
       -20.69565217, -20.08695652, -19.47826087, -18.86956522,
       -18.26086957, -17.65217391, -17.04347826, -16.43478261,
       -15.82608696, -15.2173913 , -14.60869565, -14.        ,
       -13.39130435, -12.7826087 , -12.17391304, -11.56521739,
       -10.95652174, -10.34782609,  -9.73913043,  -9.13043478,
        -8.52173913,  -7.91304348,  -7.30434783,  -6.69565217,
        -6.08695652,  -5.47826087,  -4.86956522,  -4.26086957,
        -3.65217391,  -3.04347826,  -2.43478261,  -1.82608696,
        -1.2173913 ,  -0.60869565,   0.        ])

In [28]:
programs=list()
for idx4, dwell in enumerate(dwells):
    for idx1, power in enumerate(powers):
        for idx2, Z in enumerate(Zs):
            idx = (idx1*len(powers)+idx2)
            X=Xs[idx]
            Y=Ys[idx3+idx4+1]
            print("{} {} {:.4f}".format(X, Y, Z))

1 9.0 -28.0000
2 9.0 -27.3913
3 9.0 -26.7826
4 9.0 -26.1739
5 9.0 -25.5652
6 9.0 -24.9565
7 9.0 -24.3478
8 9.0 -23.7391
9 9.0 -23.1304
10 9.0 -22.5217
11 9.0 -21.9130
12 9.0 -21.3043
13 9.0 -20.6957
14 9.0 -20.0870
15 9.0 -19.4783
16 9.0 -18.8696
17 9.0 -18.2609
18 9.0 -17.6522
19 9.0 -17.0435
20 9.0 -16.4348
21 9.0 -15.8261
22 9.0 -15.2174
23 9.0 -14.6087
24 9.0 -14.0000
25 9.0 -13.3913
26 9.0 -12.7826
27 9.0 -12.1739
28 9.0 -11.5652
29 9.0 -10.9565
30 9.0 -10.3478
31 9.0 -9.7391
32 9.0 -9.1304
33 9.0 -8.5217
34 9.0 -7.9130
35 9.0 -7.3043
36 9.0 -6.6957
37 9.0 -6.0870
38 9.0 -5.4783
39 9.0 -4.8696
40 9.0 -4.2609
41 9.0 -3.6522
42 9.0 -3.0435
43 9.0 -2.4348
44 9.0 -1.8261
45 9.0 -1.2174
46 9.0 -0.6087
47 9.0 0.0000
4 9.0 -28.0000
5 9.0 -27.3913
6 9.0 -26.7826
7 9.0 -26.1739
8 9.0 -25.5652
9 9.0 -24.9565
10 9.0 -24.3478
11 9.0 -23.7391
12 9.0 -23.1304
13 9.0 -22.5217
14 9.0 -21.9130
15 9.0 -21.3043
16 9.0 -20.6957
17 9.0 -20.0870
18 9.0 -19.4783
19 9.0 -18.8696
20 9.0 -18.2609
21 9.0 -1

In [29]:
programs=list()
for idx4, dwell in enumerate(dwells):
    for idx1, power in enumerate(powers):
        for idx2, Z in enumerate(Zs):
            idx = (idx1*len(powers)+idx2)
            X=Xs[idx]
            Y=Ys[idx3+idx4+1]
            cfg = {
                "X": X,
                "Y": Y,
                "Z": Z,
                "power": power,
                "dwell": dwell,
                "machine": cnc
            }
            program = dot(**cfg)
            programs.append(program)

In [30]:
Ys

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
       14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26.,
       27., 28., 29., 30., 31., 32., 33., 34., 35., 36., 37., 38., 39.,
       40., 41., 42., 43., 44., 45., 46., 47., 48., 49., 50., 51., 52.,
       53., 54., 55., 56., 57., 58., 59., 60., 61., 62., 63., 64., 65.,
       66., 67.])

In [31]:
import time

In [32]:
for idx, program in enumerate(programs):
    print("{}: ".format(idx), end="")
    t1=time.time()
    out = cnc.run(program)
    t2=time.time()
    print("{}/{} {}".format(out, t2-t1, (out/t2-t1)))

0: 5.299824953079224/5.299861431121826 -1539498099.9286938
1: 1.6925163269042969/1.6925458908081055 -1539498105.2294927
2: 1.69279146194458/1.6928203105926514 -1539498106.9229593
3: 1.6928684711456299/1.6928982734680176 -1539498108.615993
4: 1.6926844120025635/1.6927123069763184 -1539498110.3090994
5: 1.692887544631958/1.692927360534668 -1539498112.002466
6: 1.6931428909301758/1.693173885345459 -1539498113.6956093
7: 1.6927587985992432/1.6927871704101562 -1539498115.388993
8: 1.6929054260253906/1.692936897277832 -1539498117.081999
9: 1.6941418647766113/1.6941711902618408 -1539498118.7757401
10: 1.692380666732788/1.692417860031128 -1539498120.4707265
11: 1.692883014678955/1.6929128170013428 -1539498122.1639512
12: 1.6926958560943604/1.692725419998169 -1539498123.857665
13: 1.6932876110076904/1.6933162212371826 -1539498125.5512075
14: 1.6929795742034912/1.6930131912231445 -1539498127.2454717
15: 1.6925041675567627/1.6925337314605713 -1539498128.9394386
16: 1.6924552917480469/1.6924855709

134: 1.6932437419891357/1.693272352218628 -1539498338.882872
135: 1.6927516460418701/1.6927802562713623 -1539498340.576347
136: 1.69266939163208/1.6927039623260498 -1539498342.2699454
137: 1.6927123069763184/1.6927454471588135 -1539498343.9636056
138: 1.6929023265838623/1.6929330825805664 -1539498345.6571548
139: 1.6940503120422363/1.694082260131836 -1539498347.350301
140: 1.6922755241394043/1.6923058032989502 -1539498349.0453436
141: 6.4993064403533936/6.499338626861572 -1539498350.7384577
142: 1.6932411193847656/1.6932697296142578 -1539498357.2380254
143: 1.6934711933135986/1.6935081481933594 -1539498358.932114
144: 1.6929042339324951/1.6929447650909424 -1539498360.6264634
145: 1.6927995681762695/1.6928279399871826 -1539498362.3196187
146: 1.6933174133300781/1.693352222442627 -1539498364.012698
147: 1.6936371326446533/1.6936683654785156 -1539498365.706295
148: 1.692519187927246/1.6925482749938965 -1539498367.4008648
149: 1.6937391757965088/1.6937811374664307 -1539498369.0944147
150: 

266: 1.6921055316925049/1.6921463012695312 -1539498575.6647499
267: 1.6928644180297852/1.692894697189331 -1539498577.3577049
268: 1.69254469871521/1.692573070526123 -1539498579.050806
269: 1.694383144378662/1.69441556930542 -1539498580.7443266
270: 1.693228006362915/1.6932566165924072 -1539498582.4393368
271: 1.6920700073242188/1.6920976638793945 -1539498584.1328082
272: 1.69254469871521/1.6925735473632812 -1539498585.825115
273: 1.6931102275848389/1.693138837814331 -1539498587.5179167
274: 1.6923587322235107/1.6923882961273193 -1539498589.211994
275: 1.6933250427246094/1.6933529376983643 -1539498590.904594
276: 1.6934709548950195/1.6935031414031982 -1539498592.5981963
277: 1.6924259662628174/1.6924548149108887 -1539498594.2919128
278: 1.6927378177642822/1.6927690505981445 -1539498595.9845843
279: 1.6938683986663818/1.693896770477295 -1539498597.6775575
280: 1.6928951740264893/1.6929233074188232 -1539498599.3716595
281: 1.6919631958007812/1.692000150680542 -1539498601.0655482
282: 6.44

SerialException: Could not configure port: (5, 'Input/output error')

In [ ]:
programs[0]